In [108]:
from astropy.io import fits
from astropy import wcs
import bz2
import os
import urllib.request
import numpy as np

In [287]:
def asymmetry(ra, dec, r, petrorad, expab, expphi, run, camcol, field, rowf, colf, FITS_directory='Fits files/'  ,rerun=301):
    #Returns an array of the fourier A1 parameter and the phase angle for a given object in the SDSS field specified
    #by run, rerun, camcol, field, row, col @ row, col.
    #r= Radius (in number of petrosian radiuses) to calculate the fourier coefficient for
    #petrorad=the petrosian radius of the object in arcseconds
    #expphi = The angle of the exponential fit, expressed as degrees, N through E
    #expAB= The ratio of the semi major axes of the disk
    # Run, rerun, camcol, field = Specified the SDSS field the object is contained in
    # rowf, colf = the pixel coordinates of the centre of the image, in FITS coordinates (ie, first pixel is 1, not zero) 
    # FITS_directory=Location of folder of fits files
    #Open FITS file from disk
    try:
        #Generate the name the FITS file will be stored under
        runstring='0'*(6-len(str(run)))+str(run) 
        fieldstring='0'*(4-len(str(field)))+str(field)
        fitsname='frame-i-'+runstring+'-'+str(camcol)+'-'+fieldstring
        #Try to open the fits file. If it throws a filenotfound error, it's because this field hasn't been downloaded yet
        file=fits.open(FITS_directory+fitsname+'.fits')
    except FileNotFoundError:
        #Generate the URL the zipped FITS file will be stored under
        fitsurl='http://data.sdss3.org/sas/dr12/boss/photoObj/frames/'+str(rerun)+'/'+str(run)+'/'+str(camcol)+'/'+fitsname+'.fits.bz2'
        #Save the zipped FITS file, extract the unzipped fits file
        urllib.request.urlretrieve(fitsurl, FITS_directory+fitsname+'.fits.bz2')
        f=bz2.open(FITS_directory+fitsname+'.fits.bz2')
        #Save the unzipped FITS file to disk
        save=fits.open(f)
        save.writeto(FITS_directory+fitsname+'.fits')
        save.close()
        #Remove the zipped file to save space
        os.remove(FITS_directory+fitsname+'.fits.bz2')
        file=fits.open(FITS_directory+fitsname+'.fits')
    image=file[0] 
    data=image.data

    #Convert centre row and column to python convention (starting at zero), instead of FITS convention(starting at 1)
    rowc=rowf-1
    colc=colf-1
    
    
    #Get coordinate parameters from header using astropy WCS class. 
    coords=wcs.WCS(image.header)
    
    #Checks ra & dec agree with FITS header to within 1 pixel. 
    checkradec(ra, dec, coords, rowc, colc)
    
    #Set up sub-function to return row, col pairs for given ra, dec values
    def pix(raf,decf):
        #Takes values of ra and dec (in degrees). Converts to an array of [row, col] in pixel values
        array=coords.all_world2pix(raf,decf,0)
        return array[::-1]
    

    #Get length of semi-major and semi-minor axes of petrosian ellipse. Assume petrosian radius is the geometric mean of a & b. In degrees
    a=(2./(1+expab**2))**0.5*petrorad/3600.
    b=(2./(1+expab**-2))**0.5*petrorad/3600.
    
    #Get semi-major and semi-minor axes of the ellipse to calculate the fourier coefficients around. In arcsec
    ar=a*r
    br=b*r
    
    
    #Create an array of theta values to sample the ellipse at
    dtheta=360./360.
    thetas=np.arange(0,360,dtheta)
    rs=getrs(thetas, ar, br, expphi)
    
    #Create array with data values for each theta, r combination
    pixbrightness=[]
    for i in range(len(thetas)):
        thetarad=thetas[i]*2*np.pi/360.
        #Calculate ra, dec values of target
        #First calculate distances from centre
        drai=rs[i]*np.sin(thetarad)
        ddeci=rs[i]*np.cos(thetarad)
        #Then add on coordinates of centre
        rai=ra+drai
        deci=dec+ddeci
        
        #Convert to pixels
        pixel=pix(rai,deci)
        rowi=float(pixel[0])
        coli=float(pixel[1])
        rowiround=int(round(rowi))
        coliround=int(round(coli))
        pixbrightness.append(data[rowiround, coliround])

    
    file.close()
    
    
    
def ellipse(a,b,theta,phi):
    #returns the r coordinate of the ellipse with:
    #semi major axes a,b at polar angle theta degrees (measured from north, through east),
    #with semi-major axis offset from north by phi degrees (measured from north through east)
    thetarad=theta*2*np.pi/360.
    phirad=phi*2*np.pi/360.
    r=a*b/((b*np.cos(thetarad-phirad))**2+(a*np.sin(thetarad-phirad))**2)**0.5
    return r

def checkradec(ra, dec, coords, row, col):
    #checks to see if the pixel coordinates returned by a WCS object coords match ra and dec to correct values row and column
    reportedrow=coords.all_world2pix(ra,dec,0)[1]
    reportedcol=coords.all_world2pix(ra,dec,0)[0]
    try:
        if (reportedcol-col)**2>1 or (reportedrow-row)**2>1:
            raise ValueError('Coordinates from PhotoObj and ra & dec values do not match')
    except ValueError:
        raise
def getrs(thetas, a, b, phi):
    rs=[]
    for i in thetas:
        rs.append(ellipse(a,b,i,phi))
    return rs
        

In [286]:
asymmetry(211.6933,-0.7880827,1,11.10158,0.7288934,67.95884,752,2,455,855.3524,543.3588)

[0.096069336, 0.096069336, 0.072753906, 0.072753906, 0.034118652, 0.034118652, 0.034118652, 0.018676758, 0.018676758, 0.041870117, -0.0045318604, 0.065063477, 0.065063477, 0.010955811, 0.010955811, 0.049621582, 0.049621582, 0.049621582, -0.043212891, -0.043212891, -0.027709961, 0.13476562, 0.13476562, 0.13476562, 0.12695312, 0.12695312, 0.18115234, 0.18115234, 0.28955078, 0.28955078, 0.54492188, 0.54492188, 0.78417969, 1.8066406, 1.6425781, 1.6425781, 0.72265625, 0.72265625, 0.29736328, 0.29736328, 0.12719727, 0.12719727, 0.072998047, 0.072998047, 0.034240723, 0.034240723, -0.019897461, -0.019897461, 0.026550293, 0.026550293, 0.026550293, 0.11938477, 0.11938477, 0.080688477, 0.049743652, 0.026550293, 0.018829346, 0.018829346, 0.049804688, -0.027587891, -0.027587891, 0.049804688, 0.049804688, 0.042053223, 0.072998047, 0.034301758, 0.034301758, 0.026580811, 0.057556152, 0.057556152, 0.057556152, 0.10400391, 0.10400391, 0.065307617, 0.034301758, 0.026580811, 0.026580811, 0.018859863, 0.08

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [99]:
ellipse(1,0.5,185,5)

1.0

[4 5 6]
